<a href="https://colab.research.google.com/github/MadalinaStefan/OCR_comparison/blob/main/KerasOcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I sistemi di riconoscimento ottico dei caratteri: confronto tra Keras Ocr, Tesseract e Easy Ocr**

### Installazione e importazione librerie

La fase preliminare prevede l'installazione e l'importazione delle librerie necessarie.

In [1]:
!pip install keras_ocr
!pip install pybind11
!pip3 install fastwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastwer: filename=fastwer-0.1.3-cp310-cp310-linux_x86_64.whl size=827756 sha256=8607f4d891042ffd02ed834418e94ed97f4dfffe0e98eff27e7da687f7bdce66
  Stored in directory: /root/.cache/pip/wheels/99/b5/22/f6edc9a914bf3b54ebc55e258e2f9aa2297908d85799de228f
Successfully built fastwer


Il dataset di immagini e i file .csv conteneti il golden text sono situati e vengono importati da google Drive.

In [2]:
from google.colab import drive
import os
import keras_ocr
import matplotlib.pyplot as plt
import pandas as pd
import fastwer

drive.mount('/content/drive')
pipeline = keras_ocr.pipeline.Pipeline()

Mounted at /content/drive
Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5


### Acquisizione delle immagini

Il codice seguente è stato utilizzato per associare a ciascuna immagine (contenuta nel dataset immagini), il corrispettivo testo golden. Vengono create inizialmente tre liste vuote che verranno popolate con le immagini, le cartelle e il testo golden.
Ad ogni elemento presente nel dataset delle immagini, divise per tipologia, viene associato il corrispondente file .csv che contiene i testi golden di quella determinata categoria di immagini. L'associazione viene effetuata attraverso la corrispondenza dei nomi delle cartelle del dataset e dei nomi dei file .csv.
In *images_folder* vengono caricati di volta in volta i dataset nella qualità e nella trasformazione che si vuole visualizzare.


In [3]:
images_folder = '/content/drive/MyDrive/dataset_immagini_high_natural' # Il dataset_immagini viene aggiornato ogni volta con la qualità e la trasformazione
csv_folder = '/content/drive/MyDrive/golden_text'

images = []
labels = []
true_texts = []

for folder_class in os.listdir(images_folder):
  path = csv_folder + '/' + folder_class + '.csv'
  texts = pd.read_csv(path).values


  for file in os.listdir(images_folder + '/' + folder_class):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
      images.append(images_folder + '/' + folder_class + '/' + file)
      labels.append(folder_class)
      index = int(file.split('.')[0])
      true_texts.append(texts[index][0])


In [4]:
import numpy as np

Le immagini vengono lette attraverso keras_ocr e successivamente convertite in array.

In [5]:
images_path = images
images = [keras_ocr.tools.read(img) for img in images]
images = np.array(images, dtype=object)
images_path = np.array(images_path, dtype=object)
true_texts = np.array(true_texts, dtype=object)

# **KERAS-OCR**


### Il riconoscimento ottico dei caratteri

Per il riconoscimento del testo contenuto in ciascuna immagine si utilizza una pipeline per ottenere le previsioni dei caratteri riconosciuti. I testi predetti vengono aggiunti ad una stringa temporanea e successivamente aggiunti alla lista *pred_texts*.

In [6]:
pred_texts = []

for image in images:
  predictions = pipeline.recognize([image])

  for prediction in predictions:
    tmp = ''

    for text, box in prediction:
        print(f'Testo: {text}')
        if tmp == '':
          tmp = tmp + text
        else:
          tmp = tmp + ' ' + text

    pred_texts.append(tmp)

1/1 [==============================] - 3s 3s/step
Testo: we
Testo: ise
Testo: by
Testo: ing
Testo: li
Testo: f
Testo: ris
Testo: t
Testo: he
1/1 [==============================] - 1s 561ms/step
Testo: right
Testo: lane
Testo: must
Testo: turn
Testo: right
Testo: acans
Testo: s
Testo: s
Testo: cit
Testo: nofly
Testo: s
Testo: vn
1/1 [==============================] - 0s 317ms/step
Testo: a
Testo: gh
Testo: a
1/1 [==============================] - 0s 30ms/step
Testo: stop
Testo: right
Testo: turn
Testo: only
Testo: maneri
Testo: 92
Testo: 150
Testo: property
Testo: city
Testo: df
Testo: los
Testo: angerls
1/1 [==============================] - 0s 293ms/step
Testo: one
Testo: way
1/1 [==============================] - 0s 353ms/step
Testo: 
Testo: 1957
Testo: 300
Testo: sl
1/1 [==============================] - 0s 25ms/step
Testo: 443
Testo: nee
1/1 [==============================] - 0s 27ms/step
Testo: 7c0
Testo: 2780
Testo: cz
1/1 [==============================] - 0s 28ms/step
Testo: 34

In [7]:
from PIL import Image

La funzione *ocr_on_keras* esegue il riconoscimento ottico dei caratteri su una serie di immagini all'interno di una cartella. Il testo di ogni gruppo di previsioni viene reccolto in una stringa e successivamente tutte le stringhe di ogni immagine vengono raggruppate nella lista pred_texts.

In [8]:
from re import L

def ocr_on_keras(images):
    pipeline = keras_ocr.pipeline.Pipeline()

    pred_texts = []

    for image_path in images:
        image = keras_ocr.tools.read(image_path)
        prediction_groups = pipeline.recognize([image])
        text = ' '.join([word_info[0] for word_info in prediction_groups[0]])
        pred_texts.append(text)

    return pred_texts


### Metriche di valutazione



La seguente funzione calcola i punteggi CER e WER dei risultati prodotti dal programma, prendendo in cosiderazione e confrontanto la lista dei testi predetti con la lista dei testi golden (forniti in precedenza dall'utente).

In [9]:
def calculate_scores(pred_texts, true_texts):
  scores_cer = [ fastwer.score_sent(pred_texts[i].lower(), true_texts[i].lower(), char_level=True) for i in range(len(pred_texts)) ]

  scores_wer = [ fastwer.score_sent(pred_texts[i].lower(), true_texts[i].lower()) for i in range(len(pred_texts)) ]

  return scores_cer, scores_wer

I risultati CER e WER vengono visualizzati in file .csv per categoria di immagine. Mostrano la trasformazione eseguita e i risultati ottenuti.  

In [10]:
def generate_csv(data_class, trans, cer, wer):
  d = {'augmentation': trans, 'cer': cer, 'wer': wer}
  df = pd.DataFrame(data=d)

  csv_name = data_class + '.csv'
  df.to_csv(csv_name, sep=',', index=False)
  df

Viene definita una seconda funzione per l'implementazione dei file .csv contenenti il percorso dell'immagine, il testo golden e il testo predetto. Questa funzione è interessante per poter visualizzare i caratteri che vengono riconosciuti dal programma, inoltre può risultare utile per comprendere il funzionamento delle operazioni CER  e WER e verificare che tra le coppie formate da testo golden e testo predetto ci sia corrispondenza.

In [11]:
def generate_text_csv(csv_label, files_path, true_text, pred_texts):
  d = {'filename': files_path, 'true_text': true_text, 'pred_text': pred_texts}
  df = pd.DataFrame(data=d)
  csv_name = csv_label + '.csv'
  df.to_csv(csv_name, sep=',', index=False)
  return df

### Risultati

Ogni immagine all'interno di una categoria di immagini viene analizzata per la qualità e la trasformazione del dataset che ha ricevuto nella fase iniziale. Vengono create le colonne dei file .csv e vengono invocate le due funzioni create precedentemente per la creazione dei file dei risultati.
I risultati CER e WER illustrano la media ottenuta dal riconoscimento di tutte le immagini di una determinata classe.

In [12]:
labels = np.array(labels)

for label in set(labels):
  print(label)
  class_index = np.where(labels==label)[0]
  test_images = []

  column_cer = []
  column_wer = []
  column_trans = []

  test_images = images[class_index]

  pred_texts = ocr_on_keras(test_images)

  cer, wer = calculate_scores(pred_texts, true_texts[class_index])

  column_cer.append(round(np.mean(cer), 2))
  column_wer.append(round(np.mean(wer), 2))
  column_trans.append("high natural") # Specifiche delle caratteristiche del dataset

      # Permette di visualizzare il testo predetto e il testo golden
  #generate_text_csv(label, images_path[class_index], true_texts[class_index], pred_texts)

      # Genera un file .csv per ogni label e permette di visualizzare i risultati CER e WER
  generate_csv(label, column_trans, column_cer, column_wer)

unstructured_text
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 26ms/step
structured_text
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


3/3 [==============================] - 0s 24ms/step
handwritten_texts
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 0s 29ms/step
car_plates
Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 0s 41ms/step
